# 01 - Build Final Dataset

Merge all raw CSV files into one master dataset

In [4]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append('../src')
from data_pipeline import load_raw_data, merge_datasets, handle_missing_values, build_final_dataset

## Load All Raw Data

In [ ]:
# Load all raw files using the pipeline function
try:
    dfs = load_raw_data('../data/raw')
    print(f"[OK] Total files loaded: {len(dfs)}")
except Exception as e:
    print(f"[ERROR] Failed to load raw data: {e}")
    raise

NameError: name 'load_raw_data' is not defined

## Sales LONG format 

In [2]:
# Check actual column names in each dataset
print("📋 ACTUAL COLUMN NAMES IN EACH DATASET:\n")
for name, df in dfs.items():
    print(f"{name.upper()}:")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Shape: {df.shape}")
    print()

📋 ACTUAL COLUMN NAMES IN EACH DATASET:



NameError: name 'dfs' is not defined

## Data Quality Checks

In [4]:
# Let's look at sample data from key files
print("\n📊 SAMPLE DATA:\n")
print("SALES (main transaction data):")
print(dfs['sales'].head(3))
print("\nSTORES (store info):")
print(dfs['stores'].head())


📊 SAMPLE DATA:

SALES (main transaction data):


NameError: name 'dfs' is not defined

## Merge Datasets

In [5]:
# ✓ Data is ALREADY in long format (not wide format)
# Each row = one transaction (date, item_id, store_id, quantity, price)
# So we DON'T need to melt/pivot

print("\n" + "="*70)
print("✓ DATA FORMAT CHECK")
print("="*70)
print(f"Data is already in LONG FORMAT ✓")
print(f"  No need for pivot/melt operations")
print(f"  Each row = 1 transaction on 1 date")
print(f"  We can directly merge on (item_id, store_id, date)")
print("="*70)


✓ DATA FORMAT CHECK
Data is already in LONG FORMAT ✓
  No need for pivot/melt operations
  Each row = 1 transaction on 1 date
  We can directly merge on (item_id, store_id, date)


## Save Final Dataset

In [6]:
# Use the complete pipeline to build final dataset
print("\n" + "="*70)
print("🚀 BUILDING MASTER DATASET FROM ALL 8 CSV FILES")
print("="*70 + "\n")

final_df = merge_datasets(dfs)
final_df = handle_missing_values(final_df)

# Save to processed folder
os.makedirs('../data/processed', exist_ok=True)
final_df.to_csv('../data/processed/final_dataset.csv', index=False)

print(f"\n✅ Master dataset built and saved!")
print(f"📊 Final dataset shape: {final_df.shape}")
print(f"📁 Saved to: ../data/processed/final_dataset.csv")
print("\n" + "="*70)


🚀 BUILDING MASTER DATASET FROM ALL 8 CSV FILES



NameError: name 'merge_datasets' is not defined